## Snapista Demo notebook - Creating a binning operator and running it in a graph...

In [1]:
# Some information...

import os
import sys
print(sys.version)

import sysconfig
print("Location of esa_snappy package: " + sysconfig.get_paths()['purelib'] + os.sep + "esa_snappy")

3.11.9 | packaged by Anaconda, Inc. | (main, Apr 19 2024, 16:40:41) [MSC v.1916 64 bit (AMD64)]
Location of esa_snappy package: D:\olaf\miniconda3_py311\Lib\site-packages\esa_snappy


In [2]:
# Import esa_snappy and snapista...

import esa_snappy
import snapista

from snapista.binning import aggregators
from snapista.binning import BinningVariable, BinningOutputBands, BinningBand, Aggregators, BinningVariables
from snapista import Graph
from snapista import Operator
import lxml.etree as etree
import os

In [3]:
# Create an Aggregator for band 'IWV' (integrated water vapour column) in input products...

ag = aggregators.AggregatorAvg(varName='IWV', targetName="iwv", outputCounts='true', outputSums='true', weightCoeff=1.0)

In [4]:
# Create a binning operator. Apply to band 'IWV'...

binning = Operator('Binning')

binning.sourceProductFormat = 'NetCDF4-BEAM'

binning.aggregatorConfigs = Aggregators([ag,
                                        aggregators.AggregatorAvgOutlier(varName='IWV', targetName='outlier'),
                                         aggregators.AggregatorMinMax(varName='IWV', targetName='iwv'),
                                        aggregators.AggregatorOnMaxSet(targetName='omvn', onMaxVarName='IWV', setVarNames='IWV'),
                                        aggregators.AggregatorPercentile(varName='IWV', percentage=90, targetName='percent'),
                                        aggregators.AggregatorSum(varName='IWV', targetName='sum')])

binning.sourceProductPaths = './data/subset_1_of_S3A_OL_2*.nc'

binning.outputFile = './data/snapista_demo_binning_result.nc'
binning.outputFormat = 'NetCDF4-BEAM'

In [5]:
binning

Operator('Binning', sourceProductPaths='./data/subset_1_of_S3A_OL_2*.nc', sourceProductFormat='NetCDF4-BEAM', sourceGraphPaths='None', region='None', startDateTime='None', periodDuration='None', timeFilterMethod='NONE', minDataHour='None', numRows='2160', superSampling='1', maxDistanceOnEarth='-1', maskExpr='None', variableConfigs='None', aggregatorConfigs='<snapista.binning.aggregators.Aggregators object at 0x000001DCE25FA2D0>', postProcessorConfig='None', outputType='Product', outputFile='./data/snapista_demo_binning_result.nc', outputFormat='NetCDF4-BEAM', bandConfigurations='None', productCustomizerConfig='None', outputBinnedData='false', outputTargetProduct='true', metadataPropertiesFile='./metadata.properties', metadataTemplateDir='.', metadataAggregatorName='NAME', planetaryGridClass='org.esa.snap.binning.support.SEAGrid')

In [6]:
# Set up a graph...
g = Graph()

# Add binning operator to the graph
g.add_node(operator=binning, 
           node_id='binning_1')

aggregatorConfigs <snapista.binning.aggregators.Aggregators object at 0x000001DCE25FA2D0>
Instance TargetBandDescriptors: False
Instance Aggregators: True
Instance BinningOutputBands: False
Instance BinningVariables: False
Instance str: False
bandConfigurations None
postProcessorConfig None
productCustomizerConfig None
variableConfigs None


In [7]:
# View graph in XML representation...

g.view()

<graph>
  <version>1.0</version>
  <node id="binning_1">
    <operator>Binning</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <aggregators>
        <aggregator>
          <type>AVG</type>
          <varName>IWV</varName>
          <targetName>iwv</targetName>
          <weightCoeff>1.0</weightCoeff>
          <outputCounts>true</outputCounts>
          <outputSums>true</outputSums>
        </aggregator>
        <aggregator>
          <type>AVG_OUTLIER</type>
          <varName>IWV</varName>
          <targetName>outlier</targetName>
        </aggregator>
        <aggregator>
          <type>MIN_MAX</type>
          <varName>IWV</varName>
          <targetName>iwv</targetName>
        </aggregator>
        <aggregator>
          <type>ON_MAX_SET</type>
          <targetName>omvn</targetName>
          <onMaxVarName>IWV</onMaxVarName>
          <setVarNames>IWV</setVarNames>
        </aggregator>
        <aggregator>
          <type>PERCEN

In [8]:
# Save graph as XML file, then run it...

g.save_graph(filename='./data/snapista_demo_binning.xml')
g.run()

Processing the graph, this may take a while. Please wait...
Executing processing graph
45%90% done.
SEVERE: org.esa.snap.runtime.Engine: Failed to modify class loader field 'usr_paths'
java.lang.NoSuchMethodException: java.lang.ClassLoader.initializePath(java.lang.String)
	at java.base/java.lang.Class.getDeclaredMethod(Class.java:2848)
	at org.esa.snap.runtime.Engine.setJavaLibraryPath(Engine.java:286)
	at org.esa.snap.runtime.Engine.setJavaLibraryPath(Engine.java:265)
	at org.esa.snap.runtime.Engine.<init>(Engine.java:46)
	at org.esa.snap.runtime.Engine.start(Engine.java:117)
	at org.esa.snap.runtime.Engine.start(Engine.java:90)
	at org.esa.snap.runtime.Launcher.run(Launcher.java:51)
	at org.esa.snap.runtime.Launcher.main(Launcher.java:31)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)
	at java.base/java.lang.reflect.Method.invoke(Method.java:580)
	at com.exe4j.runtime.LauncherEngine.launch(LauncherEngine.java:84)
	at com.exe4

0